In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import sys
sys.path.append('../')
from src.config import config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'pandas'

In [3]:
raw_df = pd.read_excel()

PosixPath('/Users/56325/Documents/GitHub/data')